# 交互式回测分析页面
这个回测分析页面可以帮助您快速测试和优化量化交易策略。
支持选择不同策略、配置参数、执行回测并可视化结果。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, clear_output
import time

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 简单量化框架类
class SimpleQuantFramework:
    
    def __init__(self, symbol, strategy):
        self.symbol = symbol
        self.strategy = strategy
        self.initial_capital = 1000000  # 初始资金
        self.equity_curve = []
    
    def run_backtest(self):
        # 生成模拟数据
        np.random.seed(42)
        days = 252
        returns = np.random.normal(0.0005, 0.02, days)
        
        # 根据策略调整收益率
        if self.strategy == '多均线策略':
            returns *= 1.2  # 模拟更好的表现
        
        # 计算资金曲线
        capital = self.initial_capital
        self.equity_curve = [capital]
        
        for r in returns:
            capital *= (1 + r)
            self.equity_curve.append(capital)
        
        # 计算性能指标
        total_return = (self.equity_curve[-1] / self.initial_capital - 1) * 100
        
        # 计算夏普率 (假设无风险收益率为0)
        daily_returns = np.diff(self.equity_curve) / self.equity_curve[:-1]
        sharpe_ratio = np.sqrt(252) * daily_returns.mean() / daily_returns.std()
        
        # 计算最大回撤
        running_max = np.maximum.accumulate(self.equity_curve)
        drawdown = (self.equity_curve - running_max) / running_max
        max_drawdown = drawdown.min() * 100
        
        return {
            'total_return': total_return,
            'sharpe_ratio': sharpe_ratio,
            'max_drawdown': max_drawdown,
            'equity_curve': self.equity_curve
        }

# 回测分析UI类
class BacktestAnalyzerUI:
    
    def __init__(self):
        self.widgets = {}
        self.create_widgets()
        self.create_layout()
        self.setup_callbacks()
    
    def create_widgets(self):
        # 策略选择
        self.widgets['strategy'] = widgets.Dropdown(
            options=['简单均线交叉策略', '多均线策略'],
            value='简单均线交叉策略',
            description='策略:',
            disabled=False
        )
        
        # 合约代码输入
        self.widgets['symbol'] = widgets.Text(
            value='IF2312',
            placeholder='输入合约代码',
            description='合约:',
            disabled=False
        )
        
        # 运行按钮
        self.widgets['run_button'] = widgets.Button(
            description='运行回测',
            button_style='success',
            tooltip='点击运行回测',
            icon='play'
        )
        
        # 结果输出区域
        self.widgets['output'] = widgets.Output()
    
    def create_layout(self):
        # 创建主布局
        self.main_layout = widgets.VBox([
            self.widgets['strategy'],
            self.widgets['symbol'],
            self.widgets['run_button'],
            self.widgets['output']
        ])
    
    def setup_callbacks(self):
        # 设置按钮回调
        self.widgets['run_button'].on_click(self.on_run_backtest)
    
    def on_run_backtest(self, button):
        # 执行回测
        with self.widgets['output']:
            clear_output()
            
            # 获取用户输入
            symbol = self.widgets['symbol'].value
            strategy = self.widgets['strategy'].value
            
            print('开始回测...')
            print('策略:', strategy)
            print('合约:', symbol)
            
            # 模拟进度条
            for i in range(1, 101):
                time.sleep(0.01)
                if i % 20 == 0:
                    print(f'进度: {i}%')
            
            # 创建并运行回测框架
            framework = SimpleQuantFramework(symbol, strategy)
            results = framework.run_backtest()
            
            # 显示结果
            print('
回测结果:')
            print(f'总收益率: {results['total_return']:.2f}%')
            print(f'夏普率: {results['sharpe_ratio']:.2f}')
            print(f'最大回撤: {results['max_drawdown']:.2f}%')
            
            # 绘制资金曲线
            plt.figure(figsize=(10, 6))
            plt.plot(results['equity_curve'])
            plt.title('资金曲线图')
            plt.xlabel('交易天数')
            plt.ylabel('资金量')
            plt.grid(True)
            plt.show()
    
    def display(self):
        # 显示UI
        display(self.main_layout)

# 创建并显示回测分析页面
backtest_ui = BacktestAnalyzerUI()
backtest_ui.display()

print('交互式回测分析页面已加载完成!')